# In this Notebook we will learn how we can use different documents and get them analysed by GPT

## First of all we will install the required packages that will help us along the way
1. openai: This package will help us to call the chat completion method of openai to generate results using GPT.
3. PyMuPDF: This package is used for easy PDF manipulation.
4. tiktoken: This package is used to calculate the tokens in a text

In [ ]:
%%capture
!pip install openai PyMuPDF==1.24.2 PyMuPDFb==1.24.1 tqdm tiktoken

In [ ]:
import openai
import fitz
from tqdm import tqdm
import os
import tiktoken

In [ ]:
!pip show openai

Name: openai
Version: 1.61.1
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [ ]:
client = openai.OpenAI(api_key=" ")

model_name = "gpt-3.5-turbo"

token_encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

Here we have created a function that is called for generating response from OpenAI

This loads the keys from the environment and uses it to call the `client.chat.completions.create` method

This method takes the system message and the user message as input for generating response.

### The temperature parameter defines the randomness of the output,

Higher the temperature the more creative the LLM will become with its answers, thus higher temperatures are used for poem generation, jokes etc.
Lower temperature gives deterministic results and return the most probable next token.

### Top P

A sampling technique with temperature, called nucleus sampling, where you can control how deterministic the model is. If you are looking for exact and factual answers keep this low. If you are looking for more diverse responses, increase to a higher value. If you use Top P it means that only the tokens comprising the top_p probability mass are considered for responses, so a low top_p value selects the most confident responses. This means that a high top_p value will enable the model to look at more possible words, including less likely ones, leading to more diverse outputs.

In [ ]:
def chat_with_gpt(system_content, user_content):

    response = client.chat.completions.create(
        model= model_name,
        messages=[{"role": "system", "content": system_content},
                  {"role": "user", "content": user_content}],
        temperature=0.7,
        top_p=1.0,
    )
    return response.choices[0].message.content

## Lets take a contract and try to analyse it without much instruction

First we load the PDF and extract the texts from it and generate the token count of the text

In [ ]:
def extract_text(pdf_path):
  pdf = fitz.open(pdf_path)
  text = ''

  for page in pdf:
    text += page.get_text()

  num_tokens = len(token_encoding.encode(text))
  print("Number of tokens in the entire Document: ", num_tokens)
  return text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Out here we can see the token count of the document is 11590 which is well withing the 16000 context limit of the GPT-3.5 model

In [ ]:
short_document = extract_text("/content/drive/MyDrive/My AI Projects/Mahesh AIPM LABS/Lab-1(Generating-Response-without-RAG)/AWS1.pdf")

Number of tokens in the entire Document:  11590


## We concatenate the text from the PDF and the question that the user wants to ask to the GPT about the PDF and form a prompt that we will use to generate the response using `openai.chat.completion.create` method

In [ ]:
Question = "What is the governing courts for Amazon Web Services South Africa ProprietaryLimited"

user_content = "<Context>"+short_document+"</Context>" +"\n\n" +"<Question>"+Question+"</Question>"

In [ ]:
system_content = "You are a Professional lawyer who can analyse documents thorougly"

### We can see that the GPT was able to generate the answer by refering to the prompt and give the correct result

In [ ]:
print(chat_with_gpt(system_content, user_content))

The governing courts for Amazon Web Services South Africa Proprietary Limited are specified in the document under the "Governing Courts" section for the AWS Contracting Party for South Africa. The document states that the governing courts for Amazon Web Services South Africa Proprietary Limited are "The South Gauteng High Court, Johannesburg."

Therefore, in case of any disputes or legal matters involving Amazon Web Services South Africa Proprietary Limited, the jurisdiction for resolution would be The South Gauteng High Court in Johannesburg.


## Now lets load up a document that has more than 16000 tokens, which is the limit of GPT-3.5-Turbo

In [ ]:
long_document = extract_text("/content/drive/MyDrive/My AI Projects/Mahesh AIPM LABS/Lab-1(Generating-Response-without-RAG)/PROFRAC HOLDINGS, LLC credit agreement.pdf")

Number of tokens in the entire Document:  163227


In [ ]:
Question = "What is the Acknowledgement Regarding Any Supported QFCs?"

user_content = "<Context>"+long_document+"</Context>" +"\n\n" +"<Question>"+Question+"</Question>"

## Here what you see is, when the message length exceeded the limit of GPT, it throws an error.
### This problem will be fixed in the next lab where you see how Retrieval Augmented Generation(RAG) will fix this problem and enable us to analyse documents of any length.

In [ ]:
try:
  response = CallOpenAI(full_prompt_LD,"You are a Professional lawyer who can analyse documents thorougly")
except Exception as e:
  print(str(e))

Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 163263 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
